In [1]:
from json import load

fd = open('update_ohlcv_db.json', 'rt')
_conf = load(fd)
fd.close()
_conf

{'dbhost': 'localhost',
 'dbport': 27017,
 'db': 'rata',
 'dbcol': 'rates_5m',
 'symbol': 'EURUSD',
 'interval': 5}

In [2]:
import pandas as pd
import datetime as dt
from rata.marketon import get_data
from pymongo import MongoClient

client = MongoClient(_conf['dbhost'], _conf['dbport'])
db = client[_conf['db']]
collection = db[_conf['dbcol']]

In [3]:
if not _conf['dbcol'] in db.list_collection_names():
    hours_back = 80 * _conf['interval']
else:
    mydoc = collection.find({'symbol': 'EURUSD'}).sort('tstamp', 1).skip(collection.count_documents({}) - 9)
    df = pd.DataFrame(mydoc)
    df = df.groupby(['close', 'high', 'interval', 'low', 'open', 'status', 'symbol', 'tstamp', 'unix_tstamp', 'volume']).max('query_tstamp')
    df = df.reset_index()[['tstamp', 'interval', 'symbol', 'open', 'high', 'low', 'close', 'volume']].sort_values('tstamp')
    t1 = df['tstamp'].iloc[-1]
    t2 = dt.datetime.utcnow()
    t3 = t2 - t1
    hours_back = (t3.seconds // 3600) + 1
    print(hours_back)

df = get_data.get_finnhub(symbol=_conf['symbol'], interval=_conf['interval'], forex=True, crypto=False, hours=hours_back)
df.index = df.index.to_series().apply(dt.datetime.isoformat)
df.reset_index(inplace=True)
df_dict = df.to_dict(orient='records')


1
https://finnhub.io/api/v1/forex/candle?symbol=OANDA:EUR_USD&resolution=5&from=1638279641&to=1638283241&token=c6emgsqad3ie37m1crq0
data_status, ok
min_rates, 40
min_now, 40


In [4]:
for r in df_dict:
    #collection.update_one(r, {'$set': r}, upsert=True)
    collection.insert_one(r, {'$set': r})

In [5]:
    mydoc = collection.find({'symbol': 'EURUSD'}).sort('tstamp', 1).skip(collection.count_documents({}) - 2100)
    df = pd.DataFrame(mydoc)
    df = df.groupby(['interval',  'status', 'symbol', 'tstamp', 'unix_tstamp', ]).mean()
    df = df.reset_index()[['tstamp', 'interval', 'symbol', 'open', 'high', 'low', 'close', 'volume']].sort_values('tstamp')
    df

,tstamp,interval,symbol,open,high,low,close,volume
0,2021-11-24 23:05:00,5,EURUSD,1.12008,1.12009,1.12008,1.12008,3.0
1,2021-11-24 23:10:00,5,EURUSD,1.12008,1.12008,1.11999,1.11999,9.0
2,2021-11-24 23:15:00,5,EURUSD,1.11997,1.11998,1.11995,1.11998,3.0
3,2021-11-24 23:20:00,5,EURUSD,1.12002,1.12009,1.12000,1.12004,31.0
4,2021-11-24 23:25:00,5,EURUSD,1.12006,1.12016,1.12006,1.12015,16.0
...,...,...,...,...,...,...,...,...
1046,2021-11-30 14:15:00,5,EURUSD,1.13639,1.13666,1.13633,1.13637,397.0
1047,2021-11-30 14:20:00,5,EURUSD,1.13638,1.13666,1.13612,1.13664,416.0
1048,2021-11-30 14:25:00,5,EURUSD,1.13664,1.13709,1.13660,1.13664,421.0
1049,2021-11-30 14:30:00,5,EURUSD,1.13666,1.13736,1.13639,1.13718,764.0


In [ ]:
# predict here!